In [ ]:
print('Setup complete.')

# Lab 02: Controlling Output & Function Calling

## Learning Objectives
- Prompt an LLM to generate structured output (e.g., JSON)
- Understand the concept of function calling and how to prompt for it
- Implement a system to parse and execute function calls from an LLM
- Handle errors and validate the model's output

## Setup

In [ ]:
import json
from typing import List, Dict, Any, Optional

## Part 1: Prompting for Structured Output (JSON)

In [ ]:
class MockLLM:
    """A mock LLM that can generate JSON based on the prompt."""
    def generate(self, prompt: str) -> str:
        if 'json' in prompt.lower() and 'user' in prompt.lower():
            return '''
            ```json
            {
                "name": "John Doe",
                "age": 30,
                "email": "john.doe@example.com"
            }
            ```
            '''
        return 'I cannot provide that information as a JSON object.'

def extract_json_from_response(response: str) -> Optional[Dict[str, Any]]:
    """Extracts a JSON object from a markdown code block."""
    match = re.search(r'```json\n(.*)\n```', response, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None
    return None

llm = MockLLM()

# A prompt designed to elicit a JSON response
json_prompt = (
    'Extract the user information from the following text and provide it as a JSON object. 
'
    'The user is John Doe, he is 30 years old, and his email is john.doe@example.com.
'
    'Your response must contain only the JSON object inside a ```json code block.'
)

response = llm.generate(json_prompt)
parsed_json = extract_json_from_response(response)

print("--- Prompting for JSON ---")
print(f'LLM raw response:
{response}')
print(f'Parsed JSON object: {parsed_json}')
if parsed_json:
    print(f'Successfully extracted user name: {parsed_json.get("name")}')

## Part 2: Function Calling

Function calling allows an LLM to request that a specific function be executed in the client's code. The LLM doesn't actually run the function; it generates a structured JSON object specifying the function name and arguments. The application then parses this, runs the function, and can even return the result to the LLM.

In [ ]:
# Define the tools (functions) available to the LLM
tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

class FunctionCallingLLM:
    def generate(self, prompt: str, tools: List[Dict]) -> str:
        prompt_lower = prompt.lower()
        if 'weather' in prompt_lower and 'boston' in prompt_lower:
            # The LLM generates a JSON object representing the function call
            return json.dumps({
                "tool_calls": [
                    {
                        "type": "function",
                        "function": {
                            "name": "get_weather",
                            "arguments": '{"location": "Boston, MA"}'
                        }
                    }
                ]
            })
        return 'I can only provide weather information.'

# The actual function that exists in our application code
def get_weather(location: str) -> str:
    if 'boston' in location.lower():
        return f'The weather in {location} is 70°F and sunny.'
    return f'Weather for {location} is not available.'

## Part 3: Parsing and Executing Function Calls

In [ ]:
def execute_tool_calls(response_json: Dict, available_tools: Dict) -> Any:
    tool_calls = response_json.get('tool_calls')
    if not tool_calls:
        return "No function call requested."
    
    call = tool_calls[0]['function']
    func_name = call['name']
    func_to_call = available_tools.get(func_name)
    
    if not func_to_call:
        return f"Error: Function '{func_name}' not found."
        
    try:
        func_args = json.loads(call['arguments'])
        return func_to_call(**func_args)
    except (json.JSONDecodeError, TypeError) as e:
        return f"Error executing function: {e}"

# Create a mapping of available tool names to actual functions
available_tools = {
    "get_weather": get_weather
}

# Simulate the end-to-end process
fc_llm = FunctionCallingLLM()
user_prompt = "What's the weather like in Boston today?"

# 1. LLM generates the function call
llm_response = fc_llm.generate(user_prompt, tools=tools_schema)
response_data = json.loads(llm_response)

print("--- Function Calling Workflow ---")
print(f'LLM generated tool call: {response_data}')

# 2. Application executes the function
execution_result = execute_tool_calls(response_data, available_tools)
print(f'Result of function execution: {execution_result}')

# 3. (Optional) Return the result to the LLM for a final, natural language response
final_prompt = f'The user asked: {user_prompt}. The tool returned: {execution_result}. Formulate a natural language response.'
# final_response = llm.generate(final_prompt) -> 'The weather in Boston, MA is 70°F and sunny.'

## Exercises

1. **Add a New Tool**: Define a new tool in the `tools_schema` for a function called `send_email(to: str, subject: str, body: str)`. Implement the mock `send_email` function and update the `available_tools` mapping. Then, write a prompt that would cause the LLM to call it.
2. **Handle Multiple Function Calls**: Modify `execute_tool_calls` to handle a response from the LLM that requests multiple function calls in a single turn. The function should loop through `tool_calls` and execute each one.
3. **Implement Response Validation**: In `extract_json_from_response`, add a validation step. For example, after parsing the JSON, check if it contains the required keys (`name`, `age`, `email`). If not, return an error or `None`. You could use a library like `jsonschema` for more robust validation.

## Summary

You learned:
- How to instruct an LLM to return structured data like JSON by providing clear formatting requirements in the prompt.
- The concept of **function calling**, where an LLM generates a request to execute a function defined in your application.
- How to build a system that can parse the LLM's request, map it to an actual function, execute it, and use the result.